# Quantile Trading

## Utility Functions

In [1]:
# <include-accumulation_opportunity/utils.py>

## Imports

In [2]:
# <imports>
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.io as pio

from tqdm.notebook import trange

from accumulation_opportunity import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "none"

## Summary